In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode = True)
import us_state_abbrev
statedict = us_state_abbrev.us_state_abbrev
from pyts.preprocessing import InterpolationImputer

## Zillow Rental Index Processing

In [2]:
zori = pd.read_csv('./data/Zip_ZORI_AllHomesPlusMultifamily_SSA.csv')
zori = pd.concat([zori['RegionName'], zori.iloc[:, 4:]], axis = 1)
zori['County'] = zori['RegionName'].apply(lambda zipcode: search.by_zipcode(zipcode).values()[5][:-7]\
                                              if search.by_zipcode(zipcode).values()[5] is not None else 'NA')
zori['State'] = zori['RegionName'].apply(lambda zipcode: search.by_zipcode(zipcode).values()[6]\
                                              if search.by_zipcode(zipcode).values()[6] is not None else 'NA')
zori = pd.concat([zori[['RegionName', 'County', 'State']], zori.iloc[:, 2:-3]], axis = 1)
zori.columns = zori.columns.str.replace('RegionName', 'ZipCode')

In [4]:
temp1 = zori.iloc[:, :3].iloc[np.arange(zori.shape[0]).repeat(zori.iloc[:, 3:].shape[1])].reset_index().drop('index',
                                                                                                             axis = 1)
temp2 = pd.melt(zori.iloc[:, 3:].T.reset_index(), id_vars = 'index').drop('variable', axis = 1)
temp2.columns = ['Date', 'ZORI']
temp2['Year'] = temp2['Date'].map(lambda date: int(date[:4]))
temp2['Month'] = temp2['Date'].map(lambda date: int(date[-2:]))
temp2['Year_Month'] = temp2['Year'].map(str) + '_' + temp2['Month'].map(str)
temp2.drop('Date', axis = 1, inplace = True)
zori = pd.concat([temp1, temp2], axis = 1)

In [477]:
zori.to_csv('./data/cleandata/clean_zori.csv')

## FHFI (Home Price Index) HPI Data Processing

In [478]:
temp = pd.read_excel('./data/HPI_AT_BDL_ZIP5.xlsx', skiprows = 6)
temp = temp[temp['Year'] >= 2010]
temp = temp[['Five-Digit ZIP Code', 'Year', 'HPI with 2000 base']]
temp.columns = ['ZipCode', 'Year', 'HPI']
temp = temp[temp['HPI'] != '.']
temp['HPI'] = temp['HPI'].map(float)
temp = temp[temp['ZipCode'].isin(temp.groupby('ZipCode').count()['HPI'][temp.groupby('ZipCode').count()['HPI'] >= 8].index)]

In [480]:
temp2 = pd.DataFrame({'ZipCode': pd.unique(temp['ZipCode']).repeat(10), 
                      'Year': list(np.arange(2010, temp['Year'].max() + 1)) * len(pd.unique(temp['ZipCode']))})

In [481]:
temp = pd.merge(temp2, temp, on = ['ZipCode', 'Year'], how = 'left')

In [484]:
for zipcode, df in temp.groupby('ZipCode'):
    if df['HPI'].isnull().sum() == 0:
        continue
    else:
        imputer = InterpolationImputer()
        df['HPI'] = imputer.transform([list(df[['HPI']].reset_index()['index']), 
                                       list(df[['HPI']].reset_index()['HPI'])])[1]
        temp.loc[temp['ZipCode'] == zipcode, 'HPI'] = df['HPI']

In [13]:
yearmonth = pd.DataFrame({'Year': np.arange(temp['Year'].min(), temp['Year'].max() + 1).repeat(12),
                          'Month': list(np.arange(1, 13)) * (temp['Year'].max() - temp['Year'].min() + 1)})

In [511]:
hpi = pd.merge(temp, yearmonth, on = 'Year')
hpi['Year_Month'] = hpi['Year'].map(str) + '_' + hpi['Month'].map(str)

In [512]:
hpi = hpi[['ZipCode', 'Year', 'Month', 'Year_Month', 'HPI']]
hpi.to_csv('./data/cleandata/clean_hpi.csv')

## Zillow Home Value Index Processing

In [542]:
zhvi = pd.read_csv('./data/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv')
zhvi['CountyName'] = zhvi['CountyName'].map(lambda county: county[:-7] if county[-6:] == 'County' else county)
zhvi = pd.concat([zhvi[['RegionName', 'CountyName', 'State']], zhvi.iloc[:, 9:]], axis = 1)
zhvi.columns = zhvi.columns.str.replace('RegionName', 'ZipCode')
zhvi.columns = zhvi.columns.str.replace('CountyName', 'County')

In [543]:
temp1 = zhvi.iloc[:, :3].iloc[np.arange(zhvi.shape[0]).repeat(zhvi.iloc[:, 3:].shape[1])].reset_index().drop('index',
                                                                                                             axis = 1)
temp2 = pd.melt(zhvi.iloc[:, 3:].T.reset_index(), id_vars = 'index').drop('variable', axis = 1)
temp2.columns = ['Date', 'ZHVI']
temp2['Year'] = temp2['Date'].map(lambda date: int(date[:4]))
temp2['Month'] = temp2['Date'].map(lambda date: int(date[5:7]))
temp2['Year_Month'] = temp2['Year'].map(str) + '_' + temp2['Month'].map(str)
temp2.drop('Date', axis = 1, inplace = True)
zhvi = pd.concat([temp1, temp2], axis = 1)

In [544]:
temp3 = zhvi.groupby('ZipCode').count()['ZHVI']

In [545]:
zhvi = zhvi[zhvi['ZipCode'].isin(temp3[~temp3.where(temp3 >= 200).isnull()].index)]

In [546]:
for zipcode, df in zhvi.groupby('ZipCode'):
    if df['ZHVI'].isnull().sum() == 0:
        continue
    else:
        imputer = InterpolationImputer()
        df['ZHVI'] = imputer.transform([list(df[['ZHVI']].reset_index()['index']), 
                                        list(df[['ZHVI']].reset_index()['ZHVI'])])[1]
        zhvi.loc[zhvi['ZipCode'] == zipcode, 'ZHVI'] = df['ZHVI']

In [557]:
# Interpolation produces some negative values but for periods before 2000, which is not important for us

In [547]:
zhvi.to_csv('./data/cleandata/clean_zhvi.csv')

## Air Quality Index Data Processing

In [267]:
def airqcompiler(start, end):
    """Compiles air quality index csv files from 'start' to 'end' into 1 clean dataframe"""
    final = pd.DataFrame()
    for i in range(start, end + 1):
        temp = pd.read_csv(f'./data/daily_aqi_by_county_{i}/daily_aqi_by_county_{i}.csv')[['county Name', 'State Name',
                                                                                    'Date', 'AQI']]
        temp['State Name'] = temp['State Name'].apply(lambda state: statedict.get(state))
        temp.columns = temp.columns.str.replace('county Name', 'County')
        temp.columns = temp.columns.str.replace('State Name', 'State')
        temp['Date'] = temp['Date'].apply(lambda date: pd.to_datetime(date, format = '%Y-%m-%d'))
        temp['Month'] = temp['Date'].apply(lambda date: date.month)
        temp['Year'] = temp['Date'].apply(lambda date: date.year)
        temp = temp.groupby(['County', 'State', 'Year', 'Month']).mean().reset_index()
        
        time_df = pd.DataFrame({'Year': np.array(i).repeat(len(temp.groupby(['County', 'State']).count().index) * 12), 
                                'Month': list(np.arange(1, 13)) * len(temp.groupby(['County', 'State']).count().index)})
        time_df['County'] = sum([[county] * 12 for county, state in temp.groupby(['County', 'State']).count().index], [])
        time_df['State'] = sum([[state] * 12 for county, state in temp.groupby(['County', 'State']).count().index], [])
        
        temp = pd.merge(time_df, temp, on = ['Year', 'Month', 'County', 'State'], how = 'left')
        temp['Year_Month'] = temp['Year'].map(str) + '_' + temp['Month'].map(str)
        final = pd.concat([final, temp], axis = 0)
        print(f'Finished compiling year {i}.')
    return final

In [268]:
airq = airqcompiler(2013, 2020)

Finished compiling year 2013.
Finished compiling year 2014.
Finished compiling year 2015.
Finished compiling year 2016.
Finished compiling year 2017.
Finished compiling year 2018.
Finished compiling year 2019.
Finished compiling year 2020.


In [269]:
temp = airq.groupby(['County', 'State']).count()['AQI']

In [275]:
temp2 = pd.DataFrame({'County': [x[0] for x in temp[~temp.where(temp >= 75).isnull()].index], 
                      'State': [x[1] for x in temp[~temp.where(temp >= 75).isnull()].index]})

In [288]:
airq = pd.merge(airq, temp2, on = ['County', 'State'])

In [291]:
for (county, state), df in airq.groupby(['County', 'State']):
    if df['AQI'].isnull().sum() == 0:
        continue
    else:
        imputer = InterpolationImputer()
        df['AQI'] = imputer.transform([list(df[['AQI']].reset_index()['index']), 
                                        list(df[['AQI']].reset_index()['AQI'])])[1]
        airq.loc[(airq['County'] == county) & (airq['State'] == state), 'AQI'] = df['AQI']

In [568]:
airq.loc[airq['AQI'] < 0, 'AQI'] = 0

In [571]:
airq.to_csv('./data/cleandata/clean_airq.csv')

## Population Data Processing

In [14]:
def popcompiler(start, end):
    """Compiles population csv files from 'start' to 'end' into 1 clean dataframe"""
    final = pd.DataFrame()
    for i in range(start, end + 1):
        temp = pd.read_csv(f'./data/productDownload_2020-12-09T144241/ACSDT1Y{i}.B01003_data.csv', header = 1)
        temp = temp[['Geographic Area Name', 'Estimate!!Total']]
        temp.columns = ['County', 'Population']
        temp['State'] = temp['County'].apply(lambda county: statedict.get(county[(county.find(',') + 2):]))
        temp['County'] = temp['County'].apply(lambda county: county[:(county.find('County') - 1)] if 'County'\
                                              in county else (county[:(county.find('Municipio') - 1)] if 'Municipio'\
                                                             in county else county))
        temp['Year'] = [i for x in range(temp.shape[0])]
        final = pd.concat([final, temp], axis = 0)
    return final

In [15]:
temp = popcompiler(2011, 2019)

In [16]:
population = pd.merge(temp, yearmonth, on = 'Year')
population['Year_Month'] = population['Year'].map(str) + '_' + population['Month'].map(str)

In [518]:
population.to_csv('./data/cleandata/clean_population.csv')

## Unemployment Data Processing

In [301]:
# Clean up the unemployment dataframe
unemployment = pd.read_csv('./data/US_unemployment.csv', index_col = 0)
unemployment = unemployment[~unemployment['County'].isnull()]
unemployment['County'] = unemployment['County'] + ', ' + unemployment['State']
unemployment.drop('State', axis = 1, inplace = True)
unemployment = unemployment.T
unemployment.columns = unemployment.iloc[0, :]
unemployment = unemployment.iloc[1:, :]
unemployment.reset_index(inplace = True)
unemployment.columns.name = None
unemployment['Year'] = unemployment['index'].map(lambda year: int(year[-4:]))
unemployment = pd.concat([unemployment, pd.DataFrame(list(range(1, 13)) * int(unemployment.shape[0] / 12),
                                                     columns = ['Month'])], axis = 1)
unemployment['Year_Month'] = unemployment['Year'].map(str) + '_' + unemployment['Month'].map(str)
unemployment.drop('index', axis = 1, inplace = True)
temp = pd.DataFrame()
for county in unemployment.columns[:-3].to_list():
    temp2 = unemployment[[county, 'Year', 'Month', 'Year_Month']]
    temp2 = temp2.assign(County = temp2.columns[0])
    temp2.columns = ['Unemployment', 'Year', 'Month', 'Year_Month', 'County']
    temp = pd.concat([temp, temp2], axis = 0)
unemployment = temp
unemployment['State'] = unemployment['County'].map(lambda county: county[-2:])
unemployment['County'] = unemployment['County'].map(lambda county: county[:county.find(',')])
unemployment['Unemployment'] = unemployment['Unemployment'].map(lambda x: str(x)[:str(x).find('(')]\
                                                                if str(x).find('(') != -1 else str(x))
unemployment['Unemployment'] = unemployment['Unemployment'].map(lambda x: float(x) if (x != ' ')\
                                                                & (x != 'No Data Available ') else None)

In [438]:
temp = unemployment.groupby(['County', 'State']).count()['Unemployment']

In [447]:
temp2 = pd.DataFrame({'County': [x[0] for x in temp[~temp.where(temp >= 75).isnull()].index], 
                      'State': [x[1] for x in temp[~temp.where(temp >= 75).isnull()].index]})

In [450]:
unemployment = pd.merge(unemployment, temp2, on = ['County', 'State'])

In [472]:
unemployment.to_csv('./data/cleandata/clean_unemployment.csv')

## Education Data Processing

In [345]:
education = pd.read_csv('./data/US_education.csv').drop('Unnamed: 0', axis = 1)
education = education[education['County'] != 'United States']
education['County'] = education['County'].apply(lambda county: county[:(county.find('County') - 1)] if 'County'\
                                              in county else (county[:(county.find('Municipio') - 1)] if 'Municipio'\
                                                             in county else county))
education.loc[education['State'] == 'District of Columbia', 'State'] = 'District Of Columbia'
education['State'] = education['State'].map(lambda state: statedict.get(state))

In [346]:
education2 = pd.read_csv('./data/US_education_update.csv')
education2 = education2[education2['County'] != 'United States']
education2 = education2[education2['Year'] == 2012]
education2['County'] = education2['County'].apply(lambda county: county[:(county.find('County') - 1)] if 'County'\
                                                  in county else (county[:(county.find('Municipio') - 1)] if 'Municipio'\
                                                                 in county else county))
education2.loc[education2['State'] == 'District of Columbia', 'State'] = 'District Of Columbia'
education2['State'] = education2['State'].map(lambda state: statedict.get(state))

In [347]:
education = pd.concat([education2, education], axis = 0)

In [531]:
education = pd.merge(education, yearmonth, on = 'Year')
education['Year_Month'] = education['Year'].map(str) + '_' + education['Month'].map(str)

In [533]:
education.to_csv('./data/cleandata/clean_education.csv')

## Permits Data Processing

In [357]:
permits = pd.read_csv('./data/US_permits.csv').drop('Unnamed: 0', axis = 1)
month_to_number = {'January' : 1, 'February' : 2, 'March' : 3, 'April' : 4, 'May' : 5, 'June' : 6, 'July' : 7, 
                   'August' : 8, 'September' : 9, 'October' : 10, 'November' : 11, 'December' : 12}
permits['Month'] = permits['Month'].map(lambda month: month_to_number.get(month))
permits['Year_Month'] = permits['Year'].map(str) + '_' + permits['Month'].map(str)
temp = permits[permits['State'] == 'MN'].groupby(['Year', 'Month', 'Year_Month', 'State']).sum().reset_index()
permits = permits[permits['State'] != 'MN']
permits = pd.concat([permits, temp], axis = 0)
permits.reset_index().drop('index', axis = 1, inplace = True)

In [360]:
permits.to_csv('./data/cleandata/clean_permits.csv')

## Median Income and Total Households Data Processing

In [523]:
IandH = pd.read_csv('./data/income_and_households.csv').drop('Unnamed: 0', axis = 1)
IandH['County'] = IandH['County'].apply(lambda county: county[:(county.find('County') - 1)] if 'County'\
                                        in county else (county[:(county.find('Municipio') - 1)] if 'Municipio'\
                                                        in county else county))
IandH['Year_Month'] = IandH['Year'].map(str) + '_' + IandH['Month'].map(str)

In [525]:
IandH.to_csv('./data/cleandata/clean_IandH.csv')

## Inflation Data Processing

In [374]:
pce = pd.read_excel('./data/underlying-inflation-dashboard-data.xlsx', sheet_name = 'PCE')
pce['Year'] = pce['Date'].map(lambda date: date.year)
pce['Month'] = pce['Date'].map(lambda date: date.month)
pce.drop('Date', axis = 1, inplace = True)
pce['Year_Month'] = pce['Year'].map(str) + '_' + pce['Month'].map(str)
pce = pce[~pce['PCE'].isnull()]

In [379]:
pce.to_csv('./data/cleandata/clean_pce.csv')

## Vacancy Data Processing

In [380]:
vacancy = pd.read_csv('./data/vacancy.csv').drop('Unnamed: 0', axis = 1)
vacancy.columns = ['Year', 'County', 'State', 'Rental Vacancy Rate']

In [538]:
vacancy = pd.merge(vacancy, yearmonth, on = 'Year')
vacancy['Year_Month'] = vacancy['Year'].map(str) + '_' + vacancy['Month'].map(str)

In [540]:
vacancy.to_csv('./data/cleandata/clean_vacancy.csv')

## Job Openings Data Processing

In [391]:
jobs = pd.read_csv('./data/modified_jos.csv').drop('Unnamed: 0', axis = 1)[['State', 'Region', 'Year', 'Month', 
                                                                                  'Job Openings']]
jobs = jobs.groupby(['State', 'Year', 'Month']).sum().reset_index()
jobs['Year_Month'] = jobs['Year'].map(str) + '_' + jobs['Month'].map(str)

In [398]:
jobs.to_csv('./data/cleandata/clean_jos.csv')

## Commute and Type of Worker Data Processing

In [618]:
commute_worker = pd.read_csv('./data/commute_white_collar.csv').drop('Unnamed: 0', axis = 1)
commute_worker['County'] = commute_worker['County'].apply(lambda county: county[:(county.find('County') - 1)] if 'County'\
                                                  in county else (county[:(county.find('Municipio') - 1)] if 'Municipio'\
                                                                 in county else county))
commute_worker.drop('Selfemply', axis = 1, inplace = True)
commute_worker.drop('cChange', axis = 1, inplace = True)
commute_worker['Year_Month'] = commute_worker['Year'].map(str) + '_' + commute_worker['Month'].map(str)
commute_worker = commute_worker[['County', 'state', 'Year', 'Month', 'Year_Month', 'CommuteTime', 'Salwrkr', 'Govwrkr']]
commute_worker.columns = ['County', 'State', 'Year', 'Month', 'Year_Month', 'CommuteTime', 'Salwrkr', 'Govwrkr']

In [620]:
commute_worker.to_csv('./data/cleandata/clean_commute_worker.csv')

## GRAPI Data Processing

In [622]:
grapi = pd.read_csv('./data/grapi.csv').drop('Unnamed: 0', axis = 1)
grapi['County'] = grapi['County'].apply(lambda county: county[:(county.find('County') - 1)] if 'County'\
                                        in county else (county[:(county.find('Municipio') - 1)] if 'Municipio'\
                                                        in county else county))
grapi['Year_Month'] = grapi['Year'].map(str) + '_' + grapi['Month'].map(str)
grapi = grapi[['County', 'state', 'Year', 'Month', 'Year_Month', 'GRAPI']]
grapi.columns = ['County', 'State', 'Year', 'Month', 'Year_Month', 'GRAPI']

In [625]:
grapi.to_csv('./data/cleandata/clean_grapi.csv')

## Population Density Data Processing

In [46]:
land_area = pd.read_csv('./data/land_area.csv')
land_area['State'] = land_area['State'].map(lambda state: state[1:])
population_density = pd.merge(population, land_area, on = ['County', 'State'])
population_density['P_Density'] = population_density['Population'] / population_density['Area'] 
population_density['Year_Month'] = population_density['Year'].map(str) + '_' + population_density['Month'].map(str)
population_density = population_density[['County', 'State', 'Year', 'Month', 'Year_Month', 'P_Density']]

In [48]:
population_density.to_csv('./data/cleandata/clean_population_density.csv')

## Inequality Data Processing

In [41]:
income_inequality = pd.read_csv('./data/gini_ineq.csv').drop('Unnamed: 0', axis = 1)
income_inequality['County'] = income_inequality['County'].apply(lambda county: county[:(county.find('County') - 1)]\
                                if 'County' in county else (county[:(county.find('Municipio') - 1)] if 'Municipio'\
                                in county else county))

In [43]:
income_inequality.to_csv('./data/cleandata/clean_income_inequality.csv')